# Some DbFit for CWA things
A few simple utilities that can help with DbFit for CWA.

Helps with
1. Extracting the test part of a DbFit path, e.g. convert `http://localhost:8085/TestSuite.CwaSuite.UnitSuite.ebiz.database.code.xxlsc_am_validations_pkg` to `TestSuite.CwaSuite.UnitSuite.ebiz.database.code.xxlsc_am_validations_pkg`
2. Simple display of DbFit table data by reading from clipboard, making simple conversion to Markdown format, then displaying the Markdown
3. Way of creating the folders by invoking the usual Makefile


## 1 Extract from last /
- Reads text from clipboard
- Looks for `/`, if found, extract the text from after the `/` and copy this back to the clipboard
- Can be used to extract DbFit test details from DbFit URL by removing the `http://localhost:8085/` part, to help with command-line running.

In [13]:
import pyperclip

extracted = pyperclip.paste()
pos = extracted.rfind("/")
wanted = extracted[pos+1:]
print("Read:", extracted)

if wanted != extracted:
    pyperclip.copy(wanted)
    print("Changes made.")
    print("Now:", wanted)
    print("Result copied to clipboard")
else:
    print("No changes made.")

Read: http://localhost:8085/TestSuite.CwaSuite.UnitSuite.ebiz.database.code.xxlsc_am_validations_pkg
Changes made.
Now: TestSuite.CwaSuite.UnitSuite.ebiz.database.code.xxlsc_am_validations_pkg
Result copied to clipboard


## 2 DbFit table display
Crude way of displaying DbFit table.     
Works by converting it to Markdown tables by adding `|---` below headings row(s)

- Reads data from Clipboard and splits into lines.
- Works out the number of columns in each line by counting `|` charaters..
- If new column count found, start a new table.
- When starting a new table, add a `|---` ... row below its first row.
- Use Ipython Markdown to display the result

In [4]:
from IPython.display import display, Markdown
import pyperclip

In [8]:
# Read clipboard data
original_details = pyperclip.paste()

# Used to track if we've a new table as the number of columns should change
current_count = -1

# Break clipboard data into lines and exclude any empty ones
lines = [l for l in original_details.splitlines() if l]

# Get the number of table columns in each line
column_counts = [l.count("|") -1 for l in lines]

# Create Markdown version by adding "|---" separators when column count changes
md_lines = []
for line, count in zip(lines, column_counts):
    if count != current_count:
        md_lines.append("")
        md_lines.append(line)
        separator = "|---" * count + "|"
        md_lines.append(separator)
        current_count = count
    else:
        md_lines.append(line)
    
# Display the result
md_table = "\n".join(md_lines)
display(Markdown(md_table))



|Execute Procedure|apps.xxlsc_am_validations_pkg.civil_scope_changes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|---|---|

|p_entry_type     |p_am_bulk_file_id|p_account_number|p_area_of_law|p_item_id|p_category_id|p_submission_period|p_case_start_date  |p_am_bulk_header_id|p_ufn     |p_client_surname|p_item_name|p_am_header_id|p_am_line_id|p_schedule_ref|p_stage_reach_code|p_case_outcome_code|p_stage_level_code|p_exemption_criteria|p_ecf_reference|p_cla_reference|p_cla_exemption_code|p_client_date_of_birth|p_case_concluded_date|p_case_transfer_date|p_tolerance_indicator_flag|p_procurement_area|p_access_point|p_delivery_location|p_remote_advice|p_profit_costs|p_counsel_costs|p_tw_costs|x_invalid_outcome|x_problem_outcome|x_warning_outcome|x_error_count?|x_error_message?|x_invalid_outcome?|x_problem_outcome?|x_warning_outcome?|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|NULL             |NULL             |0Q801N          |LEGAL HELP   |177      |2155         |01/08/2018         |2022/06/01 00:00:00|NULL               |010522/123|Bloke           |IAXL:IDIF  |309           |577         |0Q801N/2018/02|NULL              |--                 |NULL              |NULL                |NULL           |NULL           |NULL                |2001/02/01 00:00:00   |2001/01/01 00:00:00  |2001/01/01 00:00:00 |NULL                      |PA30000           |AP30000       |NULL               |NULL           |300           |100            |0         |NULL             |NULL             |NULL             |NULL          |NULL            |NULL              |NULL              |NULL              |

## 3 Standard Folder and File Creator
Uses Jupyter `!` operator to run commands, so won't work as pure Python.

In [1]:
import os
import time

def nowstring():
    return time.strftime("%d/%m/%Y %H:%M:%S")

def show_results(result):
    print(nowstring())
    for r in result:
        if r:
            print(r)

def make_new_unit_test_files(make_path, package_name, procedure_name, verbose=True):
    result = !cd $test_path; make new_unit_test package=$package_name procedure=$procedure_name
    if verbose:
        show_results(result)

def make_new_feature_test_files(make_path, domain, feature, verbose=True):
    result = !cd $test_path; make new_integration_test domain=$domain feature=$feature
    if verbose:
        show_results(result)

def show_files(path):
    print(nowstring())
    unit_path = path + "/TestSuite/CwaSuite/UnitSuite/ebiz/database/code/"
    feature_path = path + "/TestSuite/CwaSuite/IntegrationSuite/"
    print("Unit", unit_path)
    !ls $unit_path
    print("Feature", feature_path)
    !ls $feature_path

### Set key parameters

In [4]:
test_path = "~/gits/laa-cwa/tests/integration"
package_name = "delete_this_pkg"
procedure_name = "delete_this_proc"

### Make unit test folders and default files

In [122]:
# Assigning result to _ just to prevent duplicate output (otherwise Jupyter auto displays but in less tidy way)
make_new_unit_test_files(test_path, package_name, procedure_name)

01/06/2022 12:01:30
==> Files created here: TestSuite/CwaSuite/UnitSuite/ebiz/database/code/delete_this_pkg/delete_this_proc
==> Start testing here: 
http://localhost:8085/TestSuite.CwaSuite.UnitSuite.ebiz.database.code.delete_this_pkg.delete_this_proc.Test1?edit


### Make feature test folders and default files

In [120]:
make_new_feature_test_files(test_path, "bad_feature", "booo")

01/06/2022 12:01:08
==> Feature files already exists...
make: *** [new_integration_test] Error 1


### Check directory contents

In [117]:
show_files(test_path)

01/06/2022 12:00:26
Unit ~/gits/laa-cwa/tests/integration/TestSuite/CwaSuite/UnitSuite/ebiz/database/code/
apps
apps.xxlsc_am_validations_pkg
apps.xxlsc_am_validations_pkg.concat_error_messages
delete_this_pkg
example_package
example_package_2
example_package_3
xxlsc_am_validations_pkg
xxlsc_am_validations_pkg.concat_error_messages
Feature ~/gits/laa-cwa/tests/integration/TestSuite/CwaSuite/IntegrationSuite/
SubmissionsSuite Validations      bad_feature      content.txt


### Original experiment
Just changing to dir and making unit test files using path, package_name and procedure defined earlier.      
Uses Jupyter `!` operator to run commands, so won't work as pure Python.

In [1]:
# Need to do in one line as ! commands don't inherit from each other (cd in one line would not have any effect on subsequent line)
!cd $test_path; make new_unit_test package=$package_name procedure=$procedure_name

make: *** No rule to make target `new_unit_test'.  Stop.
